In [1]:
# Load the packages and the Model!!
from sentence_transformers import SentenceTransformer, util
import torch
from sklearn.cluster import KMeans

# Load a pre-trained SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Fast & accurate

/home/yonas/Venvs/torch_venv/lib/python3.9/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
###########  Semantic Textual Similarity ###################
"""
Compare the meaning of two sentences.
"""

sentence1 = "A man is playing a guitar"
sentence2 = "A person is performing music"

# Encode sentences
emb1 = model.encode(sentence1, convert_to_tensor=True)
emb2 = model.encode(sentence2, convert_to_tensor=True)

# Compute cosine similarity
similarity = util.pytorch_cos_sim(emb1, emb2)
print(f"Similarity: {similarity.item():.4f}")

Similarity: 0.6417


tensor([ 2.0166e-02, -6.7581e-03, -6.7631e-03, -6.4376e-02, -1.6052e-01,
         3.6586e-04,  7.4322e-02, -2.3552e-02, -1.1231e-02,  5.8935e-02,
        -3.3138e-03,  1.4464e-02, -3.2685e-02, -4.1988e-03,  4.9535e-02,
         6.5414e-03, -4.8803e-03,  3.8985e-02,  7.9593e-02,  1.8010e-02,
         6.4069e-02,  1.1563e-01, -7.6335e-02, -5.3499e-02, -3.6927e-02,
        -6.4969e-03,  8.8151e-04,  1.5214e-01,  7.3277e-03, -2.8545e-03,
         9.6981e-02,  2.8477e-02,  8.2852e-03, -7.7340e-02, -1.2899e-01,
        -2.5143e-02, -1.3362e-01, -3.8431e-02, -1.0927e-02, -4.5478e-02,
         3.6776e-03, -2.4874e-02,  3.0344e-02,  8.6553e-02, -2.8956e-02,
        -2.6844e-02, -4.4577e-02, -1.0179e-02,  1.2767e-02, -5.0687e-02,
        -6.0824e-03,  4.8340e-02,  6.4423e-02, -6.5424e-02,  4.2060e-02,
         1.1605e-02, -2.1724e-02,  6.1911e-02, -2.4663e-02, -3.8475e-02,
        -2.8715e-02, -1.8676e-02,  1.9252e-02,  3.0594e-02,  3.9128e-02,
        -4.6102e-02,  4.6507e-02, -6.2154e-02, -2.4